# Convolutional Neural Networks

## Project: Write an Algorithm for a Dog Identification App 

---

In this notebook, some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this project. You will not need to modify the included code beyond what is requested. Sections that begin with **'(IMPLEMENTATION)'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully! 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the Jupyter Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to **File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut.  Markdown cells can be edited by double-clicking the cell to enter edit mode.

The rubric contains _optional_ "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. If you decide to pursue the "Stand Out Suggestions", you should include the code in this Jupyter notebook.



---
### Why We're Here 

In this notebook, you will make the first steps towards developing an algorithm that could be used as part of a mobile or web app.  At the end of this project, your code will accept any user-supplied image as input.  If a dog is detected in the image, it will provide an estimate of the dog's breed.  If a human is detected, it will provide an estimate of the dog breed that is most resembling.  The image below displays potential sample output of your finished project (... but we expect that each student's algorithm will behave differently!). 

![Sample Dog Output](images/sample_dog_output.png)

In this real-world setting, you will need to piece together a series of models to perform different tasks; for instance, the algorithm that detects humans in an image will be different from the CNN that infers dog breed.  There are many points of possible failure, and no perfect algorithm exists.  Your imperfect solution will nonetheless create a fun user experience!

### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Import Datasets
* [Step 1](#step1): Detect Humans
* [Step 2](#step2): Detect Dogs
* [Step 3](#step3): Create a CNN to Classify Dog Breeds (from Scratch)
* [Step 4](#step4): Create a CNN to Classify Dog Breeds (using Transfer Learning)
* [Step 5](#step5): Write your Algorithm
* [Step 6](#step6): Test Your Algorithm

---
<a id='step0'></a>
## Step 0: Import Datasets

Make sure that you've downloaded the required human and dog datasets:

**Note: if you are using the Udacity workspace, you *DO NOT* need to re-download these - they can be found in the `/data` folder as noted in the cell below.**

* Download the [dog dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip).  Unzip the folder and place it in this project's home directory, at the location `/dog_images`. 

* Download the [human dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip).  Unzip the folder and place it in the home directory, at location `/lfw`.  

*Note: If you are using a Windows machine, you are encouraged to use [7zip](http://www.7-zip.org/) to extract the folder.*

In the code cell below, we save the file paths for both the human (LFW) dataset and dog dataset in the numpy arrays `human_files` and `dog_files`.

In [1]:
# Import packages.

import numpy as np
from glob import glob
from PIL import Image
import torchvision.transforms as transforms
# from tqdm import tqdm
import torch
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from torchvision import datasets
from collections import OrderedDict
import json

# Allow loading truncated images.
from PIL import ImageFile
#ImageFile.LOAD_TRUNCATED_IMAGES = True

import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline   

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [2]:
# Check if CUDA is available.
use_cuda = torch.cuda.is_available()
use_cuda

True

In [3]:
# load filenames for human and dog images.
human_files = np.array(glob("./data/lfw/*/*"))
dog_files = np.array(glob("./data/dog_images/*/*/*"))

# Print number of images in each dataset.
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

There are 13233 total human images.
There are 8351 total dog images.


<a id='step1'></a>
## Step 1: Detect Humans

In this section, we use OpenCV's implementation of [Haar feature-based cascade classifiers](http://docs.opencv.org/trunk/d7/d8b/tutorial_py_face_detection.html) to detect human faces in images.  

OpenCV provides many pre-trained face detectors, stored as XML files on [github](https://github.com/opencv/opencv/tree/master/data/haarcascades).  We have downloaded one of these detectors and stored it in the `haarcascades` directory.  In the next code cell, we demonstrate how to use this detector to find human faces in a sample image.

In [ ]:
# extract pre-trained face detector
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')

# load color (BGR) image
img = cv2.imread(human_files[0])
# convert BGR image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# find faces in image
faces = face_cascade.detectMultiScale(gray)

# print number of faces detected in the image
print('Number of faces detected:', len(faces))

# get bounding box for each detected face
for (x,y,w,h) in faces:
    # add bounding box to color image
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    
# convert BGR image to RGB for plotting
cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# display the image, along with bounding box
plt.imshow(cv_rgb)
plt.show()

Before using any of the face detectors, it is standard procedure to convert the images to grayscale.  The `detectMultiScale` function executes the classifier stored in `face_cascade` and takes the grayscale image as a parameter.  

In the above code, `faces` is a numpy array of detected faces, where each row corresponds to a detected face.  Each detected face is a 1D array with four entries that specifies the bounding box of the detected face.  The first two entries in the array (extracted in the above code as `x` and `y`) specify the horizontal and vertical positions of the top left corner of the bounding box.  The last two entries in the array (extracted here as `w` and `h`) specify the width and height of the box.

### Write a Human Face Detector

We can use this procedure to write a function that returns `True` if a human face is detected in an image and `False` otherwise.  This function, aptly named `face_detector`, takes a string-valued file path to an image as input and appears in the code block below.

In [ ]:
# returns "True" if face is detected in image stored at img_path
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

### (IMPLEMENTATION) Assess the Human Face Detector

__Question 1:__ Use the code cell below to test the performance of the `face_detector` function.  
- What percentage of the first 100 images in `human_files` have a detected human face?  
- What percentage of the first 100 images in `dog_files` have a detected human face? 

Ideally, we would like 100% of human images with a detected face and 0% of dog images with a detected face.  You will see that our algorithm falls short of this goal, but still gives acceptable performance.  We extract the file paths for the first 100 images from each of the datasets and store them in the numpy arrays `human_files_short` and `dog_files_short`.

__Answer:__ 
(You can print out your results and/or write your percentages in this cell)

In [ ]:
# Identify human faces.

human_files_short = human_files[:100]
dog_files_short = dog_files[:100]

#-#-# Do NOT modify the code above this line. #-#-#

# Detect human faces in both sets.
H_human_detected = [face_detector(file) for file in human_files_short]
D_human_detected = [face_detector(file) for file in dog_files_short]

# Print percentage of detected faces of both sets.
print(f'{100*sum(H_human_detected)/len(human_files_short):.1f}% of faces were detected in \'human_files_short\'.')
print(f'{100*sum(D_human_detected)/len(human_files_short):.1f}% of faces were detected in \'dog_files_short\'.')

We suggest the face detector from OpenCV as a potential way to detect human images in your algorithm, but you are free to explore other approaches, especially approaches that make use of deep learning :).  Please use the code cell below to design and test your own face detection algorithm.  If you decide to pursue this _optional_ task, report performance on `human_files_short` and `dog_files_short`.

---
<a id='step2'></a>
## Step 2: Detect Dogs

In this section, we use a [pre-trained model](http://pytorch.org/docs/master/torchvision/models.html) to detect dogs in images.  

### Obtain Pre-trained VGG-16 Model

The code cell below downloads the VGG-16 model, along with weights that have been trained on [ImageNet](http://www.image-net.org/), a very large, very popular dataset used for image classification and other vision tasks.  ImageNet contains over 10 million URLs, each linking to an image containing an object from one of [1000 categories](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).  

In [ ]:
# Define VGG16 model.
VGG16 = models.vgg16(pretrained=True)

# Move model to GPU if CUDA is available.
if use_cuda:
    VGG16 = VGG16.cuda()

Given an image, this pre-trained VGG-16 model returns a prediction (derived from the 1000 possible categories in ImageNet) for the object that is contained in the image.

### (IMPLEMENTATION) Making Predictions with a Pre-trained Model

In the next code cell, you will write a function that accepts a path to an image (such as `'dogImages/train/001.Affenpinscher/Affenpinscher_00001.jpg'`) as input and returns the index corresponding to the ImageNet class that is predicted by the pre-trained VGG-16 model.  The output should always be an integer between 0 and 999, inclusive.

Before writing the function, make sure that you take the time to learn  how to appropriately pre-process tensors for pre-trained models in the [PyTorch documentation](http://pytorch.org/docs/stable/torchvision/models.html).

In [ ]:
# Import ImageNet json.
imagenet = json.load(open("imagenet_class_index.json"))

# Exctract categories and replace underscores.
for key, value in imagenet.items():
    imagenet[key] = value[1].replace('_',' ')

In [ ]:
# Deifine function to predict class from ImageNet.
def VGG16_predict(img_path):
    '''
    Use pre-trained VGG-16 model to obtain index corresponding to 
    predicted ImageNet class for image at specified path
    
    Args:
        img_path: path to an image
        
    Returns:
        Index corresponding to VGG-16 model's prediction
    '''
    
    # Load image with PIL package.
    im = Image.open(img_path).convert('RGB')

    # Define transforms needed for VGG16.
    transform_im = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])])
    
    # Transform image and reshape tensor.
    im_t = transform_im(im)
    im_t = im_t.view(-1,3,224,224)
    
    # Move image to cuda.
    if use_cuda:
        im_t = im_t.cuda()
    
    # Pass image through VGG network.
    VGG16.eval()
    output = VGG16(im_t)
    
    # Get the class with highest predicted probability.
    prob, pred = torch.topk(output,1)
    
    # Exctract and return the predicted class.
    pred_class = pred.item()
    return pred_class

In [ ]:
# Display test image.
im_path = "images/Welsh_springer_spaniel_08203.jpg"
im = Image.open(im_path).convert('RGB')
plt.imshow(im)
plt.show()

# Test prediction.   
predicted_class = VGG16_predict(im_path)
print(f'The predicted category in ImageNet: {imagenet[str(predicted_class)]}.')

### (IMPLEMENTATION) Write a Dog Detector

While looking at the [dictionary](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a), you will notice that the categories corresponding to dogs appear in an uninterrupted sequence and correspond to dictionary keys 151-268, inclusive, to include all categories from `'Chihuahua'` to `'Mexican hairless'`.  Thus, in order to check to see if an image is predicted to contain a dog by the pre-trained VGG-16 model, we need only check if the pre-trained model predicts an index between 151 and 268 (inclusive).

Use these ideas to complete the `dog_detector` function below, which returns `True` if a dog is detected in an image (and `False` if not).

In [ ]:
# Deifine function to detect dogs in an image.
def dog_detector(img_path):
    
    # Get predicted class
    pred = VGG16_predict(img_path)
    
    # Return True|False if dog detected.
    return pred >= 151 and pred <= 268

In [ ]:
# Test dog detection.
print(f'Dog found in the picture: {dog_detector(im_path)}.')

### (IMPLEMENTATION) Assess the Dog Detector

__Question 2:__ Use the code cell below to test the performance of your `dog_detector` function.  
- What percentage of the images in `human_files_short` have a detected dog?  
- What percentage of the images in `dog_files_short` have a detected dog?

__Answer:__ 


In [ ]:
# Detect dogs in both sets.
H_dog_detected = [dog_detector(file) for file in human_files_short]
D_dog_detected = [dog_detector(file) for file in dog_files_short]

# Print percentage of detected dogs of both sets.
print(f'{100*sum(H_dog_detected)/len(human_files_short):.1f}% of dogs were detected in \'human_files_short\'.')
print(f'{100*sum(D_dog_detected)/len(human_files_short):.1f}% of dogs were detected in \'dog_files_short\'.')

We suggest VGG-16 as a potential network to detect dog images in your algorithm, but you are free to explore other pre-trained networks (such as [Inception-v3](http://pytorch.org/docs/master/torchvision/models.html#inception-v3), [ResNet-50](http://pytorch.org/docs/master/torchvision/models.html#id3), etc).  Please use the code cell below to test other pre-trained PyTorch models.  If you decide to pursue this _optional_ task, report performance on `human_files_short` and `dog_files_short`.

---
<a id='step3'></a>
## Step 3: Create a CNN to Classify Dog Breeds (from Scratch)

Now that we have functions for detecting humans and dogs in images, we need a way to predict breed from images.  In this step, you will create a CNN that classifies dog breeds.  You must create your CNN _from scratch_ (so, you can't use transfer learning _yet_!), and you must attain a test accuracy of at least 10%.  In Step 4 of this notebook, you will have the opportunity to use transfer learning to create a CNN that attains greatly improved accuracy.

We mention that the task of assigning breed to dogs from images is considered exceptionally challenging.  To see why, consider that *even a human* would have trouble distinguishing between a Brittany and a Welsh Springer Spaniel.  

Brittany | Welsh Springer Spaniel
- | - 
<img src="images/Brittany_02625.jpg" width="100"> | <img src="images/Welsh_springer_spaniel_08203.jpg" width="200">

It is not difficult to find other dog breed pairs with minimal inter-class variation (for instance, Curly-Coated Retrievers and American Water Spaniels).  

Curly-Coated Retriever | American Water Spaniel
- | -
<img src="images/Curly-coated_retriever_03896.jpg" width="200"> | <img src="images/American_water_spaniel_00648.jpg" width="200">


Likewise, recall that labradors come in yellow, chocolate, and black.  Your vision-based algorithm will have to conquer this high intra-class variation to determine how to classify all of these different shades as the same breed.  

Yellow Labrador | Chocolate Labrador | Black Labrador
- | -
<img src="images/Labrador_retriever_06457.jpg" width="150"> | <img src="images/Labrador_retriever_06455.jpg" width="240"> | <img src="images/Labrador_retriever_06449.jpg" width="220">

We also mention that random chance presents an exceptionally low bar: setting aside the fact that the classes are slightly imabalanced, a random guess will provide a correct answer roughly 1 in 133 times, which corresponds to an accuracy of less than 1%.  

Remember that the practice is far ahead of the theory in deep learning.  Experiment with many different architectures, and trust your intuition.  And, of course, have fun!

### (IMPLEMENTATION) Specify Data Loaders for the Dog Dataset

Use the code cell below to write three separate [data loaders](http://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) for the training, validation, and test datasets of dog images (located at `dog_images/train`, `dog_images/valid`, and `dog_images/test`, respectively).  You may find [this documentation on custom datasets](http://pytorch.org/docs/stable/torchvision/datasets.html) to be a useful resource.  If you are interested in augmenting your training and/or validation data, check out the wide variety of [transforms](http://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transform)!

In [ ]:
# Test if all images from dataset can be succesfully loaded.
for index, file in enumerate(np.array(glob("/data/dog_images/train/*/*"))):
    try:
        # Try loading image.        
        Image.open(file).convert('RGB')
        print(f'Tested up to index: {index}', end='\r')
    except Exception as e:
        # Print exception with index in case of failure
        print(f'Problematic file: {index}.')
        print(f'Reason for failure: {e}!!!')

In [ ]:
# Get path to the problematic image.
truncated_file = np.array(glob("/data/dog_images/*/*/*"))[2271]
print(f'Corrupted file path: \'{truncated_file}\'.')

In [ ]:
# Allow loading truncated images.
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# Display truncated image.
print('Such a lovely dog has to stay in the training dataset!!! :)')
im = Image.open(truncated_file).convert('RGB')
plt.imshow(im)
plt.show()

# Test if predicting on truncated file works.   
predicted_class = VGG16_predict(truncated_file)
print(f'The predicted class coresponding to category in ImageNet: {imagenet[str(predicted_class)]}.')

In [ ]:
# Paths to train, validation and test sets.
root_dir = './data/dog_images/'
train_dir = root_dir + 'train/'
valid_dir = root_dir + 'valid/'
test_dir = root_dir + 'test/'

In [ ]:
# Get mean and std for training dataset.
# Compose transformations.
calculate_trans = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()])

# Define image loader.
calculate_data = datasets.ImageFolder(train_dir, transform = calculate_trans)
calculate_loader = torch.utils.data.DataLoader(calculate_data, batch_size=1, shuffle=False, num_workers=0)

In [ ]:
# Lists to collect means and stds for all images.
means = list()
stds = list()

# Calculate mean and std for each image
for batch, (image, label) in enumerate(calculate_loader):
    arr = np.array(image)
    mean = np.mean(arr, axis = (0,2,3))
    std = np.std(arr, axis = (0,2,3))
    means.append(mean)
    stds.append(std)

In [ ]:
# Get mean of means.
print(f'The mean of training dataset: {np.mean(means, axis = 0)}')

# Get mean of stds.
print(f'The std of training dataset: {np.mean(stds, axis = 0)}')

In [ ]:
# Define transformations for train, validation and test sets.
train_trans_sc = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),    
    transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])
    #transforms.Normalize([0.48640606,  0.45601946,  0.39183509],[ 0.23047173,  0.22563872,  0.22351009])
    #transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

test_valid_trans_sc = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])
    #transforms.Normalize([0.48640606,  0.45601946,  0.39183509],[ 0.23047173,  0.22563872,  0.22351009])
    #transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [ ]:
# Dataloaders with local paths.
train_data_sc = datasets.ImageFolder(train_dir, transform = train_trans_sc)
valid_data_sc = datasets.ImageFolder(valid_dir, transform = test_valid_trans_sc)
test_data_sc = datasets.ImageFolder(test_dir, transform = test_valid_trans_sc)

collect_data = [train_data_sc, valid_data_sc, test_data_sc]

In [ ]:
# Define dataloaders for train, validation and test sets.
train_loader_sc = torch.utils.data.DataLoader(train_data_sc, batch_size=10, shuffle=True, num_workers=0)
valid_loader_sc = torch.utils.data.DataLoader(valid_data_sc, batch_size=10, shuffle=True, num_workers=0)
test_loader_sc = torch.utils.data.DataLoader(test_data_sc, batch_size=10, shuffle=False, num_workers=0)

loaders_scratch = {'train':train_loader_sc,'valid':valid_loader_sc,'test':test_loader_sc}

In [ ]:
# Investigate data
print(f'Number of classes: {len(np.array(glob(train_dir + "*")))}.')
print(f'Number of examples: \nTrain set: {len(train_data_sc)}, \nValidation set: {len(valid_data_sc)}, \nTest set: {len(test_data_sc)}.')

In [ ]:
# Show transformations applied on training data
practice_loader = torch.utils.data.DataLoader(train_data_sc, batch_size=1, shuffle=True)

def show_transformations():
    data, target = next(iter(practice_loader))
    arr = np.array(data)
    image = arr.reshape(3,224,224).transpose(1,2,0)
    image = image*np.array([0.5, 0.5, 0.5]) + np.array([0.5, 0.5, 0.5])
    image[image < 0] = 0
    image[image > 1] = 1
    plt.imshow(image)
    plt.show()

show_transformations()

In [ ]:
# del VGG16

**Question 3:** Describe your chosen procedure for preprocessing the data. 
- How does your code resize the images (by cropping, stretching, etc)?  What size did you pick for the input tensor, and why?
- Did you decide to augment the dataset?  If so, how (through translations, flips, rotations, etc)?  If not, why not?


**Answer**:<br>
Preprocessing data is achieved with *torchvision.transforms* module. I choose to start with similar preprocessing that I used while writing function *VGG16_predict* by resizing picture to 256 pixels on the shorter side and cropping out 224x224 pixels from the center of the image. I added two transformations to increase the models accuracy, random rotation and random horizontal flip. After transforming image to a tensor, the values must be normalized with mean and standard deviation for each color channel. I tried different approaches to do so. First was using the same values, that are required for pretrained networks trained on ImageNet dataset. Second was calculating the mean and std of the training dataset in this project. With both of these methods I was not able to achieve accuracy above 10%. The final approach was to normalize values with *mean = 0.5* and *std = 0.5* for all color channels, which transforms values to be within the range \[-1,1\].

### (IMPLEMENTATION) Model Architecture

Create a CNN to classify dog breed.  Use the template in the code cell below.

In [ ]:
# Define the CNN architecture.
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        
        # Define convolutional layers
        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding = 1)
        self.conv5 = nn.Conv2d(256, 512, 3, padding = 1)
        
        # Max pooling layer
        self.pool1 = nn.MaxPool2d(2,2)
        
        # Define fully connected layers
        self.fc1 = nn.Linear(512*7*7, 500)
        self.fc2 = nn.Linear(500, 500)
        self.fc3 = nn.Linear(500, 133)
        
        # Define dropout for fully connected layers
        self.dropout = nn.Dropout(0.5)        
    
    def forward(self, x):
        # Pass through conv. layers
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool1(F.relu(self.conv2(x)))
        x = self.pool1(F.relu(self.conv3(x)))
        x = self.pool1(F.relu(self.conv4(x)))        
        x = self.pool1(F.relu(self.conv5(x)))
        
        # Flatten x.
        x = x.view(-1,512*7*7)
        
        # Pass through fc layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)        
        return x

#-#-# You do NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()
print(model_scratch)

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

__Question 4:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  

__Answer:__ <br>
The final version of the architecture has five convolutional layers starting with 32 filters and ending with 512. Each filter has *size = 3*, *stride = 1* and *padding = 1*. After each layer a *ReLU* activation function is applied, followed by max pooling layer with *kernel size = 2* and *stride = 2*, that decreases the size of image by half.
Convolutional layers are followed by three fully connected layers starting with 25088 input nodes and ending with 133 output nodes, which corresponds to the number of dog breed classes. First two layers are followed by *ReLU* activation function and dropout with probability *p = 0.5*. Last fully connected layer *fc3* has no activation function.
With training in 30 epochs this simple architecture reaches accuracy of 18.18% on test set.

### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/stable/optim.html).  Save the chosen loss function as `criterion_scratch`, and the optimizer as `optimizer_scratch` below.

In [ ]:
# Select loss function.
criterion_scratch = nn.CrossEntropyLoss()

# Select optimizer.
optimizer_scratch = optim.Adam(model_scratch.parameters(), lr=0.001)

In [4]:
# difference between memory_cached and memory_allocated: https://discuss.pytorch.org/t/memory-allocated-vs-memory-cached/32365
print(f'memory allocated {torch.cuda.memory_allocated()/1e9:.2f} Giga-bytes')
print(f'memory allocated {torch.cuda.memory_cached()/1e9:.2f} Giga-bytes')

memory allocated 0.00 Giga-bytes
memory allocated 0.00 Giga-bytes


C:\Users\markein\Anaconda3\lib\site-packages\torch\cuda\memory.py:344: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_scratch.pt'`.

In [5]:
# Define function to train a model.
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        print(f'memory allocated {torch.cuda.memory_allocated()/1e9:.2f} Giga-bytes')
        print(f'memory allocated {torch.cuda.memory_cached()/1e9:.2f} Giga-bytes')
        
        ###################
        # train the model #
        ###################
        model.train()
        
        print(f'\nTraining epoch {epoch}...', end='\r')
        for batch_idx, (data, target) in enumerate(loaders['train']):    

            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
                
            # Set gradients to 0
            optimizer.zero_grad()            
            # Forward pass
            output = model(data)
            # Calculate loss
            loss = criterion(output,target)
            # Backward pass
            loss.backward()
            # take optimizer step
            optimizer.step()
            # Get average training loss            
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
       
        print(f'Evaluating epoch {epoch}...', end='\r')
        
        
        
        correct = 0.
        total = 0.
        
        
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # Forward pass
            with torch.no_grad():
                output = model(data)
            # Calculate loss
            loss = criterion(output,target)
            # Get average validation loss            
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            # convert output probabilities to predicted class
            pred = output.data.max(1, keepdim=True)[1]
            # compare predictions to true label
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total += data.size(0)           

            
        # print training/validation statistics 
        print(f'Epoch: {epoch}.. Train Loss: {train_loss:.3f}... Valid Loss: {valid_loss:.3f}... Valid Accuracy: {100.*correct/ total:.2f}% ({correct:.0f}/{total:.0f})...') 
        #print(f'\nTraining Accuracy: {100.*train_accuracy/ train_total:.2f}% ({train_accuracy:.0f}/{train_total:.0f})')
        #print(f'\n')
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print(f'...In epoch {epoch} the validation loss decreased from {valid_loss_min:.6f} to {valid_loss:.6f}. Model saved...')
            # Save current state dict
            torch.save(model.state_dict(),save_path)
            # Update min. validation loss
            valid_loss_min = valid_loss
            
            
    # return trained model
    return model



In [ ]:
# Train the model.
model_scratch = train(3, loaders_scratch, model_scratch, optimizer_scratch, criterion_scratch, use_cuda, 'model_scratch.pt')

In [ ]:
print(f'memory allocated {torch.cuda.memory_allocated()/1e9:.2f} Giga-bytes')
print(f'memory allocated {torch.cuda.memory_cached()/1e9:.2f} Giga-bytes')

In [ ]:
# Optionaly save the model even if validation loss did not decrease.
#torch.save(model_scratch.state_dict(),'model_scratch.pt')

In [ ]:
# load the model that got the best validation accuracy.
model_scratch.load_state_dict(torch.load('model_scratch.pt'))

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images.  Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 10%.

In [6]:
# Define function to ge test accuracy.
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # Forward pass
        with torch.no_grad():
            output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    #print('Test Loss: {:.6f}\n'.format(test_loss))

    print(f'\nTest Accuracy: {100.*correct/ total:.2f}% ({correct:.0f}/{total:.0f})')

In [ ]:
# Call test function.  
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

---
<a id='step4'></a>
## Step 4: Create a CNN to Classify Dog Breeds (using Transfer Learning)

You will now use transfer learning to create a CNN that can identify dog breed from images.  Your CNN must attain at least 60% accuracy on the test set.

### (IMPLEMENTATION) Specify Data Loaders for the Dog Dataset

Use the code cell below to write three separate [data loaders](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader) for the training, validation, and test datasets of dog images (located at `dogImages/train`, `dogImages/valid`, and `dogImages/test`, respectively). 

If you like, **you are welcome to use the same data loaders from the previous step**, when you created a CNN from scratch.

In [7]:
print(f'memory allocated {torch.cuda.memory_allocated()/1e9:.2f} Giga-bytes')
print(f'memory allocated {torch.cuda.memory_cached()/1e9:.2f} Giga-bytes')

memory allocated 0.00 Giga-bytes
memory allocated 0.00 Giga-bytes


In [8]:
# Allow loading truncated images.
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [9]:
# Paths to train, validation and test sets.
root_dir = './data/dog_images/'
train_dir = root_dir + 'train/'
valid_dir = root_dir + 'valid/'
test_dir = root_dir + 'test/'

In [10]:
# Define transformations for train, validation and test sets.
train_trans_tr = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])])

test_valid_trans_tr = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])])

In [11]:
# Dataloaders with local paths.
train_data_tr = datasets.ImageFolder(train_dir, transform = train_trans_tr)
valid_data_tr = datasets.ImageFolder(valid_dir, transform = test_valid_trans_tr)
test_data_tr = datasets.ImageFolder(test_dir, transform = test_valid_trans_tr)

data_transfer = {'train':train_data_tr, 'valid':valid_data_tr, 'test':test_data_tr}

In [12]:
# Define dataloaders for train, validation and test sets.
train_loader_tr = torch.utils.data.DataLoader(train_data_tr, batch_size=20, shuffle=True, num_workers=0)
valid_loader_tr = torch.utils.data.DataLoader(valid_data_tr, batch_size=20, shuffle=True, num_workers=0)
test_loader_tr = torch.utils.data.DataLoader(test_data_tr, batch_size=20, shuffle=False, num_workers=0)

loaders_transfer = {'train':train_loader_tr,'valid':valid_loader_tr,'test':test_loader_tr}

### (IMPLEMENTATION) Model Architecture

Use transfer learning to create a CNN to classify dog breed.  Use the code cell below, and save your initialized model as the variable `model_transfer`.

In [13]:
# Use VGG16 for transfer learning.
model_transfer = models.vgg16(pretrained=True)

# Freeze parameters of the model.
for param in model_transfer.parameters():
    param.requires_grad = False

# Print out classifier.
model_transfer.classifier   

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [14]:
# Define new classifier.
dog_breed_classifier = nn.Sequential(OrderedDict([
    ('0', nn.Linear(25088,4096)),
    ('1', nn.ReLU()),
    ('2', nn.Dropout(0.5)),
    ('3', nn.Linear(4096,4096)),
    ('4', nn.ReLU()),
    ('5', nn.Dropout(0.5)),
    ('6', nn.Linear(4096,133))
        ]))

In [15]:
# Change classifier in the model.
model_transfer.classifier = dog_breed_classifier

In [16]:
# Move model to GPU if CUDA is available.
if use_cuda:
    model_transfer = model_transfer.cuda()

__Question 5:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  Describe why you think the architecture is suitable for the current problem.

__Answer:__ <br>
VGG16 seems to be like a good easy option to implement for transfer learning. Since VGG NNs were trained on ImageNet dataset, they can identify 1000 classes of various objects, from which more than 100 are different dog breeds. Our dataset has 133 dog breeds, so it can be expected that this NN should be able to extract most features about specific to each of the dog breeds reasonably well.<br>
To make things easier I decided to modify only the number of output nodes from the classifier, so that it matches the number of classes in our dataset. Just this minor change and training on 5 epochs results in accuracy of 70.93%.


### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/master/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/master/optim.html).  Save the chosen loss function as `criterion_transfer`, and the optimizer as `optimizer_transfer` below.

In [17]:
# Select loss function.
criterion_transfer = nn.CrossEntropyLoss()

# Select optimizer, only optimize parameters of the classifier.
optimizer_transfer = optim.Adam(model_transfer.classifier.parameters(), lr=0.001)

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_transfer.pt'`.

In [18]:
# Train the model.
model_transfer = train(10, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

memory allocated 0.54 Giga-bytes
memory allocated 0.56 Giga-bytes

Epoch: 1.. Train Loss: 5.173... Valid Loss: 4.202... Valid Accuracy: 12.34% (103/835)...
...In epoch 1 the validation loss decreased from inf to 4.202414. Model saved...
memory allocated 1.99 Giga-bytes
memory allocated 2.88 Giga-bytes

Epoch: 2.. Train Loss: 5.218... Valid Loss: 4.282... Valid Accuracy: 10.78% (90/835)...
memory allocated 1.99 Giga-bytes
memory allocated 2.88 Giga-bytes

Epoch: 3.. Train Loss: 5.444... Valid Loss: 4.467... Valid Accuracy: 6.47% (54/835)...
memory allocated 1.99 Giga-bytes
memory allocated 2.88 Giga-bytes

Epoch: 4.. Train Loss: 5.260... Valid Loss: 4.469... Valid Accuracy: 7.54% (63/835)...
memory allocated 1.99 Giga-bytes
memory allocated 2.88 Giga-bytes

Epoch: 5.. Train Loss: 5.257... Valid Loss: 4.497... Valid Accuracy: 7.07% (59/835)...
memory allocated 1.99 Giga-bytes
memory allocated 2.88 Giga-bytes

Epoch: 6.. Train Loss: 5.220... Valid Loss: 4.646... Valid Accuracy: 8.02% (67/

In [19]:
# Optionaly save the model even if validation loss did not decrease.
torch.save(model_transfer.state_dict(),'model_transfer.pt')

In [ ]:
# load the model that got the best validation accuracy.
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images. Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 60%.

In [ ]:
# Call test function.  
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

### (IMPLEMENTATION) Predict Dog Breed with the Model

Write a function that takes an image path as input and returns the dog breed (`Affenpinscher`, `Afghan hound`, etc) that is predicted by your model.  

In [ ]:
### TODO: Write a function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.

# list of class names by index, i.e. a name can be accessed like class_names[0]
class_names = [item[4:].replace("_", " ") for item in data_transfer['train'].classes]

def predict_breed_transfer(img_path):
        # Load image with PIL package.
    im = Image.open(img_path).convert('RGB')

    # Define transforms needed for VGG16.
    transform_im = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])])
    
    # Transform image and reshape tensor.
    im_t = transform_im(im)
    im_t = im_t.view(-1,3,224,224)
    
    if use_cuda:
        im_t = im_t.cuda()
    
    # Pass image through VGG network.
    model_transfer.eval()
    with torch.no_grad():
        output = model_transfer(im_t)
        
        # Get the class with highest predicted probability.
        prob, pred = torch.topk(output,1)
    
    # Exctract and return the predicted class.
    pred_class = pred.item()
    dog = class_names[pred_class]
    
    return pred_class, dog

---
<a id='step5'></a>
## Step 5: Write your Algorithm

Write an algorithm that accepts a file path to an image and first determines whether the image contains a human, dog, or neither.  Then,
- if a __dog__ is detected in the image, return the predicted breed.
- if a __human__ is detected in the image, return the resembling dog breed.
- if __neither__ is detected in the image, provide output that indicates an error.

You are welcome to write your own functions for detecting humans and dogs in images, but feel free to use the `face_detector` and `human_detector` functions developed above.  You are __required__ to use your CNN from Step 4 to predict dog breed.  

Some sample output for our algorithm is provided below, but feel free to design your own user experience!

![Sample Human Output](images/sample_human_output.png)


### (IMPLEMENTATION) Write your Algorithm

In [ ]:
# Define app to identify dog breeds.
def run_app(img_path):
    ## handle cases for a human face, dog, and neither
    if dog_detector(img_path):
        print('\nI definitely see a dog in this picture.')
        
        # Let's take a look at the picture
        im = Image.open(img_path).convert('RGB')
        plt.imshow(im)
        plt.show()
        
        # Predict dog breed.
        index, breed_class = predict_breed_transfer(img_path)
        print(f'I think it is a {breed_class}.')
        if breed_class == 'Australian shepherd':
            print('!!!The best dog ever!!!')
    
    
    elif face_detector(img_path):
        print('\nThis must be a human.')
 
        # Predict dog breed.
        index, breed_class = predict_breed_transfer(img_path)
        
        # Get image of predicted breed
        folder = data_transfer['train'].classes[index]
        dog = np.array(glob(f"/data/dog_images/train/{folder}/*"))[0]
        
        # Let's take a look at the picture
        im, (ax1, ax2) = plt.subplots(1, 2, figsize = (10,10))
        im1 = Image.open(img_path).convert('RGB')
        ax1.imshow(im1)
        im2 = Image.open(dog).convert('RGB')
        ax2.imshow(im2)
        plt.show()        

        print(f'...Who looks like a {breed_class}. Do they look alike?')        
        
        
    else:
        print('\nI don\'t see any dogs or humans here!!!')
        # Let's take a look at the picture
        im = Image.open(img_path).convert('RGB')
        plt.imshow(im)
        plt.show()
        
        # Predict VGG16 category
        ImageNet_class = VGG16_predict(img_path)
        category = imagenet[str(ImageNet_class)]
        
        print(f'This looks more like a {category}...')

---
<a id='step6'></a>
## Step 6: Test Your Algorithm

In this section, you will take your new algorithm for a spin!  What kind of dog does the algorithm think that _you_ look like?  If you have a dog, does it predict your dog's breed accurately?  If you have a cat, does it mistakenly think that your cat is a dog?

### (IMPLEMENTATION) Test Your Algorithm on Sample Images!

Test your algorithm at least six images on your computer.  Feel free to use any images you like.  Use at least two human and two dog images.  

__Question 6:__ Is the output better than you expected :) ?  Or worse :( ?  Provide at least three possible points of improvement for your algorithm.

__Answer:__ <br>
The model seems to be doing decent job in identifying dog breeds. I was able to recognize all three Mastiffs correctly and later other tested breeds, that were part of the training set. Finding resemblance between humans and dog breeds is a little bit difficult to evaluate, but I would say I can see similar features in some cases.<br>
Some steps to improve this app could be:<br>
- 1. Add more dog breeds to the training dataset and retrain the model. Since there are more than 300 breeds recognized by [FCI](http://www.fci.be/en/) and this dataset has only 133 classes some well-known dog breeds can't be identified.<br>
- 2. Possible improvement of face detection could be done by using different [pretrained detectors](https://github.com/opencv/opencv/tree/master/data/haarcascades).<br>
- 3. Tuning of the model is another possibility. Different changes like adjusting the learning rate, increasing | decreasing the number of hidden layers, trying different optimizer or choosing different dropout can all improve the model performance.<br>
- 4. Testing different pretrained architecture instead of VGG16.
- 5. Adding more options for data augmentation. For example randomly converting images to black and gray could possibly help recognizing breeds, that have multiple color versions.

In [ ]:
# Test app on pictures from dog_files and human_files
for file in np.hstack((human_files[:3], dog_files[:3])):
    run_app(file)

In [ ]:
# Let's test a image of a ship.
run_app('./test_images/liner_ship.jpg')

In [ ]:
# What do you say about Beyonce?
run_app('./test_images/beyonce.jpg')

In [ ]:
# Let's trysome lovely Australian Shepherd.
run_app('./test_images/australian_shepherd.jpg')

In [ ]:
# Can the app recognize a hamburger?
run_app('./test_images/hamburger.jpg')

In [ ]:
# Let me introduce you to this wonderfull new dog breed, who looks like a real wolf :)
run_app('./test_images/czechoslovakian_wolfdog.jpg')

In [ ]:
# What will the app say about Prince Harry?
run_app('./test_images/prince_harry.jpg')

In [ ]:
# Finally the last dog breed tob= be tested: Alaskan malamute.
run_app('./test_images/alaskan_malamute.jpg')

*Image sources*<br>
> [Ship](https://www.stuff.co.nz/travel/news/115045145/australian-man-dies-after-falling-overboard-from-worlds-biggest-cruise-ship)<br>
> [Beyoncé](https://www.bbc.com/news/newsbeat-45082773)<br>
> [Australian Sheperd](https://www.pets4homes.co.uk/dog-breeds/australian-shepherd/)<br>
> [Hamburger](https://www.recipetineats.com/hamburger-recipe/)<br>
> [Czechoslovakian Wolfdog](https://dogsnet.com/czechoslovakian-wolfdog/)<br>
> [Prince Harry](https://happybday.to/Prince-Harry/photo?media=4513189)<br>
> [Alaskan Malamute](https://puppytoob.com/20-most-dangerous-dog-breeds/)<br>